<a href="https://colab.research.google.com/github/graceyangfan/bilibili-video-code/blob/master/mixtext.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers

     |████████████████████████████████| 1.3MB 8.6MB/s 
     |████████████████████████████████| 890kB 24.7MB/s 
     |████████████████████████████████| 1.1MB 33.7MB/s 
     |████████████████████████████████| 2.9MB 24.5MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=9e8c365d052c391250b1363c22be82325828272db00adfa82d53365b23592334
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
!pip install boto3

     |████████████████████████████████| 133kB 8.8MB/s 
     |████████████████████████████████| 71kB 6.8MB/s 
     |████████████████████████████████| 6.9MB 13.6MB/s 
     |████████████████████████████████| 143kB 45.1MB/s 
ERROR: requests 2.23.0 has requirement urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you'll have urllib3 1.26.2 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
  Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [ ]:
!pip install Fairseq

     |████████████████████████████████| 1.7MB 8.6MB/s 
     |████████████████████████████████| 71kB 8.9MB/s 
     |████████████████████████████████| 122kB 31.5MB/s 
     |████████████████████████████████| 112kB 30.9MB/s 
     |████████████████████████████████| 276kB 32.9MB/s 
  Created wheel for antlr4-python3-runtime: filename=antlr4_python3_runtime-4.8-cp36-none-any.whl size=141230 sha256=ac26baa55b0a5e049047341d1255f4731b363c3afbc9061b72813d05f4f1ae79
  Stored in directory: /root/.cache/pip/wheels/e3/e2/fa/b78480b448b8579ddf393bebd3f47ee23aa84c89b6a78285c8
  Created wheel for PyYAML: filename=PyYAML-5.3.1-cp36-cp36m-linux_x86_64.whl size=44619 sha256=8ca57c83d85ab1eb23ccda8b4a76b589eda6d625b83e05736ba84e49c5df1090
  Stored in directory: /root/.cache/pip/wheels/a7/c1/ea/cf5bd31012e735dc1dfea3131a2d5eae7978b251083d6247bd
Successfully built antlr4-python3-runtime PyYAML
  Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
!pip install fastBPE

  Created wheel for fastBPE: filename=fastBPE-0.1.0-cp36-cp36m-linux_x86_64.whl size=481504 sha256=dba386f5746c6abe5a1252ce9c4feebeac8bafe6778198fc808a80dcb2b19a6b
  Stored in directory: /root/.cache/pip/wheels/f3/0c/9c/fc62058b4d473a5602bcd3d3edfece796f123875379ea82d79
Successfully built fastBPE


In [ ]:
!pip install -U deep_translator

     |████████████████████████████████| 133kB 10.2MB/s 
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
  Found existing installation: urllib3 1.26.2
    Uninstalling urllib3-1.26.2:
      Successfully uninstalled urllib3-1.26.2


In [ ]:
import os
from google.colab import drive
drive.mount('/content/drive')

path = "/content/drive/My Drive/MixText/code"

os.chdir(path)
os.listdir(path)

Mounted at /content/drive


['.idea',
 '__init__.py',
 '__pycache__',
 'mixtext.py',
 'model',
 'normal_bert.py',
 'normal_train.py',
 'pytransformers',
 'read_data.py',
 'train.py',
 'build',
 'mixtext_test2.ipynb',
 'mixText.csv',
 'mixText_self_train.csv',
 'mixtext.ipynb']

In [ ]:
import pandas as pd

In [ ]:
import numpy as np
import pandas as pd
import random
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as Data
from pytransformers import *
from torch.autograd import Variable
from torch.utils.data import Dataset

import pickle
import os
import sklearn.model_selection
import sklearn.preprocessing
import joblib
from deep_translator import GoogleTranslator

In [ ]:
def get_data(data_path,nlabels,max_seq_len=256, model='bert-base-chinese', train_aug=False):
    # 加载bert的tokenizer类
    tokenizer = BertTokenizer.from_pretrained(model)

    def text_filter(sentence: str) -> str:
        """
        过滤掉非汉字和标点符号和非数字
        :param sentence:
        :return:
        """
        line = sentence.replace('\n', '。')
        # 过滤掉非汉字和标点符号和非数字
        linelist = [word for word in line if
                    word >= u'\u4e00' and word <= u'\u9fa5' or word in ['，', '。', '？', '！',
                                                                        '：'] or word.isdigit()]
        return ''.join(linelist)
    def fliter_data(data):
      data['content']=data['content'].astype(str)
      data["text"]=data["content"].apply(lambda x:text_filter(x))
      return data 

    train_df=pd.read_csv(data_path+"/train_valid.csv")
    train_ul=pd.read_csv(data_path+"/train_ul.csv")
    test_df=pd.read_csv(data_path+"/test_df.csv")

    train_df=fliter_data(train_df)
    train_ul=fliter_data(train_ul)
    test_df=fliter_data(test_df)

    total_unlabeled=pd.concat([train_ul,test_df])

    ##get train and valid set with 对抗验证
    train_labeled,valid_labeled=train_val_split(train_df,nlabels)


    train_labeled_dataset = loader_labeled(list(train_labeled["text"]),list(train_labeled["num_label"]), tokenizer, max_seq_len, train_aug)
    train_unlabeled_dataset = loader_unlabeled(list(total_unlabeled["text"]),tokenizer, max_seq_len, train_aug)
    val_dataset = loader_labeled(list(valid_labeled["text"]),list(valid_labeled["num_label"]), tokenizer, max_seq_len, train_aug)
    test_dataset = loader_unlabeled(list(test_df["text"]),tokenizer, max_seq_len, train_aug)
    

    return train_labeled_dataset, train_unlabeled_dataset, val_dataset, test_dataset, nlabels
    


In [ ]:
from sklearn.utils import shuffle

def train_val_split(data,nlabels):
    train_data=pd.DataFrame([])
    valid_data=pd.DataFrame([])
    for i in range(nlabels):
      sub_data=data[data["num_label"]==i].sort_values('is_test').reset_index(drop=True)
      train_data=pd.concat([train_data,sub_data.iloc[:int(0.8 * len(sub_data)),]])
      valid_data=pd.concat([valid_data,sub_data.iloc[int(0.8 * len(sub_data)):,]])
      train_data=shuffle(train_data)
      valid_data=shuffle(valid_data)
      
    return  train_data,valid_data

In [ ]:
class loader_unlabeled(Dataset):
    def __init__(self, dataset_text, tokenizer, max_seq_len, train_aug=False):
        """
         # 无标签数据加载器
        :param dataset_text:
        :param unlabeled_idxs:
        :param tokenizer:
        :param max_seq_len:
        :param train_aug: 是否做数据增强
        """
        self.tokenizer = tokenizer
        self.text = dataset_text
        self.train_aug = train_aug
        self.max_seq_len = max_seq_len
        #做一个空的字典，保存数据增强后的文本
        self.trans_dist = {}

    def __len__(self):
        return len(self.text)

    def augment(self, text):
        """
        数据增强, 翻译成英文，在翻译回中文
        :param text: 单个文档的文本
        :return:  新的列表，列表里面是生成后的文本
        """
        TranslatorF= GoogleTranslator(source='auto', target='en')
        TranslatorI= GoogleTranslator(source='auto', target='chinese (simplified)')
        if text not in self.trans_dist:
            text1 = TranslatorF.translate(text=text)
            text2 = TranslatorI.translate(text=text1)
            self.trans_dist[text] = text2
        return self.trans_dist[text], text

    def get_tokenized(self, text):
        """
        :param text: 接收纯文本
        :return:返回encode后的数字id，和原始长度
        """
        tokens = self.tokenizer.tokenize(text)
        if len(tokens) > self.max_seq_len:
            tokens = tokens[:self.max_seq_len]
        length = len(tokens)
        encode_result = self.tokenizer.convert_tokens_to_ids(tokens)
        padding = [0] * (self.max_seq_len - len(encode_result))
        encode_result += padding
        return encode_result, length

    def __getitem__(self, idx):
        """
        对无标签数据做数据增强与否
        :param idx:  int数字，迭代时的索引
        :return:
        """
        #如果做数据增强
        if self.train_aug:
            # 数据增强的文本augtext_u和augtext_v， 原始文本ori_text
            augtext_u, ori_text = self.augment(self.text[idx])
            encode_result_u, length_u = self.get_tokenized(augtext_u)
            encode_result_ori, length_ori = self.get_tokenized(ori_text)
            return ((torch.tensor(encode_result_u), torch.tensor(encode_result_ori)), (length_u,length_ori))
        #如果不做数据增强
        else:
            text = self.text[idx]
            encode_result, length = self.get_tokenized(text)
            return (torch.tensor(encode_result), length)

In [ ]:
class loader_labeled(Dataset):
    def __init__(self, dataset_text, dataset_label, tokenizer, max_seq_len, aug=False):
        """
        # 有标签数据的loader, trans_dist 是存储翻译后的结果，
        :param dataset_text:  文本array
        :param dataset_label:  对应的标签array
        :param tokenizer:  使用的tokenizer对象
        :param max_seq_len:  最大序列长度
        :param aug:  是否使用数据增强
        """
        self.tokenizer = tokenizer
        self.text = dataset_text
        self.labels = dataset_label
        self.max_seq_len = max_seq_len
        self.aug = aug
        self.trans_dist = {}

    def __len__(self):
        return len(self.labels)

    def augment(self, text):
        """
        数据增强, 翻译成英文，在翻译回中文
        :param text: 单个文档的文本
        :return:  新的列表，列表里面是生成后的文本
        """
        TranslatorF= GoogleTranslator(source='auto', target='en')
        TranslatorI= GoogleTranslator(source='auto', target='chinese (simplified)')
        if text not in self.trans_dist:
            text1 = TranslatorF.translate(text=text)
            text2 = TranslatorI.translate(text=text1)
            self.trans_dist[text] = text2
        return self.trans_dist[text]

    def get_tokenized(self, text):
        """
        对单个文档的文本text做tokenizer
        :param text: 纯文本内容
        :return:
        """
        tokens = self.tokenizer.tokenize(text)
        #若果大于最大长度，那么截断
        if len(tokens) > self.max_seq_len:
            tokens = tokens[:self.max_seq_len]
        #获取最后的长度
        length = len(tokens)
        # 把token 转换成id， encode_result是转换成id后的数字列表
        encode_result = self.tokenizer.convert_tokens_to_ids(tokens)
        #如果小于最长长度，那么做padding
        padding = [0] * (self.max_seq_len - len(encode_result))
        #padding 0加到末尾
        encode_result += padding

        return encode_result, length

    def __getitem__(self, idx):
        """
        :param idx: 迭代是的索引，int
        :return: 如果数据增强： 原始文本encode后的结果（做了padding），数据增强生成的文本encode后结果，各自对应的labels和未padding时的文本长度
        """
        if self.aug:
            text = self.text[idx]
            text_aug = self.augment(text)
            #对原始文本和生成的文本都进行encode， text_result是encode后的id列表，text_length是文本的实际长度
            text_result, text_length = self.get_tokenized(text)
            text_result2, text_length2 = self.get_tokenized(text_aug)
            print("augment data with google translate")
            return ((torch.tensor(text_result), torch.tensor(text_result2)), (self.labels[idx], self.labels[idx]), (text_length, text_length2))
        else:
          
            text = self.text[idx]
            encode_result, length = self.get_tokenized(text)
            #如果不做数据增强，只返回原始文本encode结果，对应label，和原始文本的长度
            return (torch.tensor(encode_result), self.labels[idx], length)


In [ ]:
class argment(object):
    def __init__(self,input_dict):
        self.epochs=input_dict["epochs"]
        self.batch_size=input_dict["batch_size"]
        self.batch_size_u=input_dict["batch_szie_u"]
        self.lrmain=input_dict["lrmax"]
        self.lrlast=input_dict["lrlast"]
        self.gpu=input_dict["gpu"]
        self.n_labeled=input_dict["n_laneled"]
        self.un_labeled=input_dict["un_labeled"]
        self.val_iteration=input_dict["val_iteration"]
        self.mix_option=input_dict["mix_option"]
        self.mix_method=input_dict["mix_method"]
        self.separate_mix=input_dict["separate_mix"]
        self.co=input_dict["co"]
        self.train_aug=input_dict["train_ug"]
        self.model=input_dict["model"]
        self.data_path=input_dict["data_path"]
        self.mix_layers_set=input_dict["mix_layers_set"]
        self.alpha=input_dict["alpha"]
        self.lambda_u=input_dict["lambda_u"]
        self.T=input_dict["T"]
        self.temp_change=input_dict["temp_change"]
        self.margin=input_dict["margin"]
        self.lambda_u_hinge=input_dict["lambda_u_hinge"]

In [ ]:
train_prameters={
    "epochs":15,
    "batch_size":4,
    "batch_szie_u":24,
    "lrmax":0.00001,
    "lrlast":0.001,
    "gpu":1,
    "n_laneled":20,
    "un_labeled":5000,
    "val_iteration":200,
    "mix_option":True,
    "mix_method":0,
    "separate_mix":False,
    "co":False,
    "train_ug":False,
    "model":'bert-base-chinese',
    "data_path":"../data/NLP",
    "mix_layers_set":[0, 1, 2, 3],
    "alpha":0.75,
    "lambda_u":1,
    "T":0.5,
    "temp_change":1000000,
    "margin":0.7,
    "lambda_u_hinge":0
}
args=argment(train_prameters)

In [ ]:
class SemiLoss(object):
    def __call__(self, outputs_x, targets_x, outputs_u, targets_u, outputs_u_2, epoch, mixed=1):
        """
        半监督损失函数
        :param outputs_x: 模型输出的x
        :param targets_x: 真实的x
        :param outputs_u: 模型输出的无标签的x
        :param targets_u:  真实的无标签的x
        :param outputs_u_2: 模型输出的无标签x_2
        :param epoch:  迭代次数
        :param mixed: 是否是混合过的输出
        :return:
        """
        if args.mix_method == 0 or args.mix_method == 1:
            #有监督的x的损失
            Lx = - \
                torch.mean(torch.sum(F.log_softmax(
                    outputs_x, dim=1) * targets_x, dim=1))
            #无监督的x输出的概率值
            probs_u = torch.softmax(outputs_u, dim=1)
            #论文中公式显示的kl散度, batch mean 批次均值作为统计计算KL散度
            Lu = F.kl_div(probs_u.log(), targets_u, None, None, 'batchmean')
            #计算hinge Loss 折页损失 max(0,1-(wTxi +b)yi)
            Lu2 = torch.mean(torch.clamp(torch.sum(-F.softmax(outputs_u, dim=1)
                                                   * F.log_softmax(outputs_u, dim=1), dim=1) - args.margin, min=0))

        elif args.mix_method == 2:
            if mixed == 0:
                Lx = - \
                    torch.mean(torch.sum(F.logsigmoid(
                        outputs_x) * targets_x, dim=1))

                probs_u = torch.softmax(outputs_u, dim=1)

                Lu = F.kl_div(probs_u.log(), targets_u,
                              None, None, 'batchmean')

                Lu2 = torch.mean(torch.clamp(args.margin - torch.sum(
                    F.softmax(outputs_u_2, dim=1) * F.softmax(outputs_u_2, dim=1), dim=1), min=0))
            else:
                Lx = - \
                    torch.mean(torch.sum(F.log_softmax(
                        outputs_x, dim=1) * targets_x, dim=1))

                probs_u = torch.softmax(outputs_u, dim=1)
                Lu = F.kl_div(probs_u.log(), targets_u,
                              None, None, 'batchmean')

                Lu2 = torch.mean(torch.clamp(args.margin - torch.sum(
                    F.softmax(outputs_u, dim=1) * F.softmax(outputs_u, dim=1), dim=1), min=0))

        return Lx, Lu, args.lambda_u * linear_rampup(epoch), Lu2, args.lambda_u_hinge * linear_rampup(epoch)

In [ ]:
def linear_rampup(current, rampup_length=args.epochs):
    if rampup_length == 0:
        return 1.0
    else:
        current = np.clip(current / rampup_length, 0.0, 1.0)
        return float(current)

In [ ]:
#only for valid 
def validate(valloader, model, criterion, epoch, mode):
    model.eval()
    with torch.no_grad():
        loss_total = 0
        total_sample = 0
        acc_total = 0
        correct = 0

        for batch_idx, (inputs, targets, length) in enumerate(valloader):
            if n_gpu !=0:
                inputs, targets = inputs.cuda(), targets.cuda(non_blocking=True)
            outputs = model(inputs)
            loss = criterion(outputs, targets)

            _, predicted = torch.max(outputs.data, 1)

            if batch_idx == 0:
                print("Sample some true labeles and predicted labels")
                print(predicted[:20])
                print(targets[:20])

            correct += (np.array(predicted.cpu()) ==
                        np.array(targets.cpu())).sum()
            loss_total += loss.item() * inputs.shape[0]
            total_sample += inputs.shape[0]

        acc_total = correct/total_sample
        loss_total = loss_total/total_sample

    return loss_total, acc_total

In [ ]:
def train(labeled_trainloader, unlabeled_trainloader, model, optimizer, scheduler, criterion, epoch, n_labels, train_aug=False):
    """
    :param labeled_trainloader: 有标签数据
    :param unlabeled_trainloader: 无标签数据
    :param model:  Mixtext 模型
    :param optimizer: 优化器
    :param scheduler: 动态更新学习率
    :param criterion: 损失函数
    :param epoch:
    :param n_labels: 标签类别数量
    :param train_aug: 是否使用数据增强
    :return:
    """
    labeled_train_iter = iter(labeled_trainloader)
    unlabeled_train_iter = iter(unlabeled_trainloader)
    model.train()
    #设定更改温度T的条件
    global total_steps
    global flag
    if flag == 0 and total_steps > args.temp_change:
        print('改变sharpen function的温度选项')
        args.T = 0.9
        flag = 1

    for batch_idx in range(args.val_iteration):

        total_steps += 1
        #如果训练集不做数据增强
        if not train_aug:
            inputs_x, targets_x, inputs_x_length = labeled_train_iter.next()
        #训练集做数据增强
        else:
            (inputs_x, inputs_x_aug), (targets_x, _), (inputs_x_length, inputs_x_length_aug) = labeled_train_iter.next()
            (inputs_u,  inputs_ori), (length_u,  length_ori) = unlabeled_train_iter.next()
        batch_size = inputs_x.size(0)
        #原始输入句子的批次
        batch_size_2 = inputs_ori.size(0)
        #targets_x 做成one_hot编码
        targets_x = torch.zeros(batch_size, n_labels).scatter_(1, targets_x.view(-1, 1), 1)
        if n_gpu != 0:
            inputs_x, targets_x = inputs_x.cuda(), targets_x.cuda(non_blocking=True)
            inputs_u = inputs_u.cuda()
            inputs_ori = inputs_ori.cuda()

        mask = []

        with torch.no_grad():
            # 对无标签数据预测标签, outputs_u是对增强的无标签数据预测，预测出的类别置信度, 首先猜测无标签数据的低熵标签
            outputs_u = model(inputs_u)
            outputs_ori = model(inputs_ori)

            # 根据不同数据翻译后的质量，给与不同的权重
            # For AG News: German: 1, Russian: 0, ori: 1
            # For DBPedia: German: 1, Russian: 1, ori: 1
            # For IMDB: German: 0, Russian: 0, ori: 1
            # For Yahoo Answers: German: 1, Russian: 0, ori: 1 / German: 0, Russian: 0, ori: 1
            p = (0 * torch.softmax(outputs_u, dim=1)  + 1 * torch.softmax(outputs_ori, dim=1)) / (1)
            # sharpen 计算，p的n次方, 如果T是0.5，那么pt就是p的平方值
            pt = p**(1/args.T)
            # targets_u 是求一个百分比,是论文中标签猜测算法部分
            targets_u = pt / pt.sum(dim=1, keepdim=True)
            targets_u = targets_u.detach()
        #是否使用mix, 1表示已经mix，是一个flag
        mixed = 1

        # 训练时是否随机选择在mix和unmix之间
        if args.co:
            mix_ = np.random.choice([0, 1], 1)[0]
        else:
            mix_ = 1
        # 如果使用mix，设置beta分布参数 beta分布的alpha参数 lbeta, lbeta是一个浮点数, beta分布的参数alpha,beta, Dirichlet 分布是由 Beta 分布推广而来的
        if mix_ == 1:
            lbeta = np.random.beta(args.alpha, args.alpha)
            if args.separate_mix:
                lbeta = lbeta
            else:
                lbeta = max(lbeta, 1-lbeta)
        else:
            lbeta = 1
        #选择哪一层进行mix_layer, 随机选择一层，减去1是为了和列表索引相对应,例如选择bert的第11层
        mix_layer = np.random.choice(args.mix_layers_set, 1)[0]
        mix_layer = mix_layer - 1

        #如果不使用数据增强
        if not train_aug:
            all_inputs = torch.cat(
                [inputs_x, inputs_u, inputs_ori, inputs_ori], dim=0)

            all_lengths = torch.cat(
                [inputs_x_length, length_u, length_ori, length_ori], dim=0)

            all_targets = torch.cat(
                [targets_x, targets_u, targets_u, targets_u, targets_u], dim=0)

        #如果使用数据增强，拼接所有输入，长度和标签（无标签数据预测出来的标签)
        else:
            all_inputs = torch.cat(
                [inputs_x, inputs_x_aug, inputs_u, inputs_ori], dim=0)
            all_lengths = torch.cat(
                [inputs_x_length, inputs_x_length, length_u, length_ori], dim=0)
            all_targets = torch.cat(
                [targets_x, targets_x, targets_u, targets_u], dim=0)
        #分别进行mix, 是使用batch_size个还是全部，是从batch_size 获取，还是从全部数据获取
        if args.separate_mix:
            #随机打乱函数randperm，获取索引，就是把无标签，有标签和数据增强的数据随机抽取出来进行训练
            idx1 = torch.randperm(batch_size)
            idx2 = torch.randperm(all_inputs.size(0) - batch_size) + batch_size
            idx = torch.cat([idx1, idx2], dim=0)

        else:
            idx1 = torch.randperm(all_inputs.size(0) - batch_size_2)
            idx2 = torch.arange(batch_size_2) + all_inputs.size(0) - batch_size_2
            idx = torch.cat([idx1, idx2], dim=0)
        #input_a是所有输入，input_b是抽取的部分输入,input和target都是embedding后的向量，length_a,length_b是原始的seq_length,未加padding时候的
        input_a, input_b = all_inputs, all_inputs[idx]
        target_a, target_b = all_targets, all_targets[idx]
        length_a, length_b = all_lengths, all_lengths[idx]

        if args.mix_method == 0:
            # Mix句子的隐藏向量,input_a是所有输入，input_b是随机抽取的输入
            logits = model(input_a, input_b, lbeta, mix_layer)
            #对target也使用Mixup混合，公式是论文上的公式
            mixed_target = lbeta * target_a + (1 - lbeta) * target_b
        elif args.mix_method == 1:
            # 拼接2个训练句子的片段，  片段的选择根据beta分布lbeta
            # 例如: "I love you so much" 和 "He likes NLP" 可能混合成 "He likes NLP so much".
            # 对应的labels也会根据系数被混合
            mixed_input = []
            if lbeta != 1:
                for i in range(input_a.size(0)):
                    length1 = math.floor(int(length_a[i]) * lbeta)
                    idx1 = torch.randperm(int(length_a[i]) - length1 + 1)[0]
                    length2 = math.ceil(int(length_b[i]) * (1-lbeta))
                    if length1 + length2 > 256:
                        length2 = 256-length1 - 1
                    idx2 = torch.randperm(int(length_b[i]) - length2 + 1)[0]
                    try:
                        if n_gpu !=0:
                            mixed_input.append(torch.cat((input_a[i][idx1: idx1 + length1], torch.tensor([102]).cuda(), input_b[i][idx2:idx2 + length2], torch.tensor([0]*(256-1-length1-length2)).cuda()), dim=0).unsqueeze(0))
                        else:
                            mixed_input.append(torch.cat((input_a[i][idx1: idx1 + length1], torch.tensor([102]), input_b[i][idx2:idx2 + length2], torch.tensor([0]*(256-1-length1-length2))), dim=0).unsqueeze(0))
                    except:
                        print(256 - 1 - length1 - length2,idx2, length2, idx1, length1)

                mixed_input = torch.cat(mixed_input, dim=0)

            else:
                mixed_input = input_a

            logits = model(mixed_input)
            mixed_target = lbeta * target_a + (1 - lbeta) * target_b

        elif args.mix_method == 2:
            # 拼接2个句子
            # 对应的label是平均值
            if lbeta == 1:
                mixed_input = []
                for i in range(input_a.size(0)):
                    if n_gpu != 0:
                        mixed_input.append(torch.cat((input_a[i][:length_a[i]], torch.tensor([102]).cuda(), input_b[i][:length_b[i]], torch.tensor([0]*(512-1-int(length_a[i])-int(length_b[i]))).cuda()), dim=0).unsqueeze(0))
                    else:
                        mixed_input.append(torch.cat((input_a[i][:length_a[i]], torch.tensor([102]), input_b[i][:length_b[i]], torch.tensor([0]*(512-1-int(length_a[i])-int(length_b[i])))), dim=0).unsqueeze(0))
                mixed_input = torch.cat(mixed_input, dim=0)
                logits = model(mixed_input, sent_size=512)

                #mixed_target = torch.clamp(target_a + target_b, max = 1)
                mixed = 0
                mixed_target = (target_a + target_b)/2
            else:
                mixed_input = input_a
                mixed_target = target_a
                logits = model(mixed_input, sent_size=256)
                mixed = 1
        #使用SemiLoss计算损失
        Lx, Lu, w, Lu2, w2 = criterion(logits[:batch_size], mixed_target[:batch_size], logits[batch_size:-batch_size_2],
                                       mixed_target[batch_size:-batch_size_2], logits[-batch_size_2:], epoch+batch_idx/args.val_iteration, mixed)
        #如果使用mix
        if mix_ == 1:
            loss = Lx + w * Lu
        else:
            loss = Lx + w * Lu + w2 * Lu2

        #max_grad_norm = 1.0
        # 梯度裁剪
        #torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        # scheduler.step()

        if batch_idx % 1000 == 0:
            print("epoch {}, step {}, loss {}, Lx {}, Lu {}, Lu2 {}".format(
                epoch, batch_idx, loss.item(), Lx.item(), Lu.item(), Lu2.item()))

In [ ]:
def train(labeled_trainloader, unlabeled_trainloader, model, optimizer, scheduler, criterion, epoch, n_labels, max_seq_len=256,train_aug=False):
    """
    :param labeled_trainloader: 有标签数据
    :param unlabeled_trainloader: 无标签数据
    :param model:  Mixtext 模型
    :param optimizer: 优化器
    :param scheduler: 动态更新学习率
    :param criterion: 损失函数
    :param epoch:
    :param n_labels: 标签类别数量
    :param train_aug: 是否使用数据增强
    :return:
    """
    labeled_train_iter = iter(labeled_trainloader)
    unlabeled_train_iter = iter(unlabeled_trainloader)
    model.train()
    #设定更改温度T的条件
    global total_steps
    global flag
    if flag == 0 and total_steps > args.temp_change:
        print('改变sharpen function的温度选项')
        args.T = 0.9
        flag = 1


    for batch_idx in range(args.val_iteration):

        total_steps += 1
        #如果训练集不做数据增强
        if not train_aug:
            inputs_x, targets_x, inputs_x_length = labeled_train_iter.next()
            inputs_ori,length_ori= unlabeled_train_iter.next()
        #训练集做数据增强
        else:
            (inputs_x, inputs_x_aug), (targets_x, _), (inputs_x_length, inputs_x_length_aug) = labeled_train_iter.next()
            (inputs_u,  inputs_ori), (length_u,  length_ori) = unlabeled_train_iter.next()
        batch_size = inputs_x.size(0)
        #原始输入句子的批次
        batch_size_2 = inputs_ori.size(0)
        #targets_x 做成one_hot编码
        targets_x = torch.zeros(batch_size, n_labels).scatter_(1, targets_x.view(-1, 1), 1)
        if n_gpu != 0:
            inputs_x, targets_x = inputs_x.cuda(), targets_x.cuda(non_blocking=True)
            inputs_ori = inputs_ori.cuda()
            if train_aug:
              inputs_u = inputs_u.cuda()

        mask = []
        with torch.no_grad():
            # 对无标签数据预测标签, outputs_u是对增强的无标签数据预测，预测出的类别置信度, 首先猜测无标签数据的低熵标签
            if train_aug:
              outputs_u = model(inputs_u)
            outputs_ori = model(inputs_ori)

            # 根据不同数据翻译后的质量，给与不同的权重
            # For AG News: German: 1, Russian: 0, ori: 1
            # For DBPedia: German: 1, Russian: 1, ori: 1
            # For IMDB: German: 0, Russian: 0, ori: 1
            # For Yahoo Answers: German: 1, Russian: 0, ori: 1 / German: 0, Russian: 0, ori: 1
            if train_aug:
              p = (0 * torch.softmax(outputs_u, dim=1)  + 1 * torch.softmax(outputs_ori, dim=1)) / (1)
            else:
              p=1 * torch.softmax(outputs_ori, dim=1) / (1)
            # sharpen 计算，p的n次方, 如果T是0.5，那么pt就是p的平方值
            pt = p**(1/args.T)
            # targets_u 是求一个百分比,是论文中标签猜测算法部分
            targets_u = pt / pt.sum(dim=1, keepdim=True)
            targets_u = targets_u.detach()
        #是否使用mix, 1表示已经mix，是一个flag
        mixed = 1

        # 训练时是否随机选择在mix和unmix之间
        if args.co:
            mix_ = np.random.choice([0, 1], 1)[0]
        else:
            mix_ = 1
        # 如果使用mix，设置beta分布参数 beta分布的alpha参数 lbeta, lbeta是一个浮点数, beta分布的参数alpha,beta, Dirichlet 分布是由 Beta 分布推广而来的
        if mix_ == 1:
            lbeta = np.random.beta(args.alpha, args.alpha)
            if args.separate_mix:
                lbeta = lbeta
            else:
                lbeta = max(lbeta, 1-lbeta)
        else:
            lbeta = 1
        #选择哪一层进行mix_layer, 随机选择一层，减去1是为了和列表索引相对应,例如选择bert的第11层
        mix_layer = np.random.choice(args.mix_layers_set, 1)[0]
        mix_layer = mix_layer - 1

        #如果不使用数据增强
        if not train_aug:
            all_inputs = torch.cat(
                [inputs_x, inputs_ori], dim=0)

            all_lengths = torch.cat(
                [inputs_x_length, length_ori], dim=0)

            all_targets = torch.cat(
                [targets_x, targets_u], dim=0)

        #如果使用数据增强，拼接所有输入，长度和标签（无标签数据预测出来的标签)
        else:
            all_inputs = torch.cat(
                [inputs_x, inputs_x_aug, inputs_u, inputs_ori], dim=0)
            all_lengths = torch.cat(
                [inputs_x_length, inputs_x_length, length_u, length_ori], dim=0)
            all_targets = torch.cat(
                [targets_x, targets_x, targets_u, targets_u], dim=0)
        #分别进行mix, 是使用batch_size个还是全部，是从batch_size 获取，还是从全部数据获取
        if args.separate_mix:
            #随机打乱函数randperm，获取索引，就是把无标签，有标签和数据增强的数据随机抽取出来进行训练
            idx1 = torch.randperm(batch_size)
            idx2 = torch.randperm(all_inputs.size(0) - batch_size) + batch_size
            idx = torch.cat([idx1, idx2], dim=0)

        else:
            idx1 = torch.randperm(all_inputs.size(0) - batch_size_2)
            idx2 = torch.arange(batch_size_2) + all_inputs.size(0) - batch_size_2
            idx = torch.cat([idx1, idx2], dim=0)
        #input_a是所有输入，input_b是抽取的部分输入,input和target都是embedding后的向量，length_a,length_b是原始的seq_length,未加padding时候的
        input_a, input_b = all_inputs, all_inputs[idx]
        target_a, target_b = all_targets, all_targets[idx]
        length_a, length_b = all_lengths, all_lengths[idx]

        if args.mix_method == 0:
            # Mix句子的隐藏向量,input_a是所有输入，input_b是随机抽取的输入
            logits = model(input_a, input_b, lbeta, mix_layer)
            #对target也使用Mixup混合，公式是论文上的公式
            mixed_target = lbeta * target_a + (1 - lbeta) * target_b
        elif args.mix_method == 1:
            # 拼接2个训练句子的片段，  片段的选择根据beta分布lbeta
            # 例如: "I love you so much" 和 "He likes NLP" 可能混合成 "He likes NLP so much".
            # 对应的labels也会根据系数被混合
            mixed_input = []
            if lbeta != 1:
                for i in range(input_a.size(0)):
                    length1 = math.floor(int(length_a[i]) * lbeta)
                    idx1 = torch.randperm(int(length_a[i]) - length1 + 1)[0]
                    length2 = math.ceil(int(length_b[i]) * (1-lbeta))
                    if length1 + length2 > max_seq_len:
                        length2 = max_seq_len-length1 - 1
                    idx2 = torch.randperm(int(length_b[i]) - length2 + 1)[0]
                    try:
                        if n_gpu !=0:
                            mixed_input.append(torch.cat((input_a[i][idx1: idx1 + length1], torch.tensor([102]).cuda(), input_b[i][idx2:idx2 + length2], torch.tensor([0]*(256-1-length1-length2)).cuda()), dim=0).unsqueeze(0))
                        else:
                            mixed_input.append(torch.cat((input_a[i][idx1: idx1 + length1], torch.tensor([102]), input_b[i][idx2:idx2 + length2], torch.tensor([0]*(256-1-length1-length2))), dim=0).unsqueeze(0))
                    except:
                        print(max_seq_len - 1 - length1 - length2,idx2, length2, idx1, length1)

                mixed_input = torch.cat(mixed_input, dim=0)

            else:
                mixed_input = input_a

            logits = model(mixed_input)
            mixed_target = lbeta * target_a + (1 - lbeta) * target_b

        elif args.mix_method == 2:
            # 拼接2个句子
            # 对应的label是平均值
            if lbeta == 1:
                mixed_input = []
                for i in range(input_a.size(0)):
                    if n_gpu != 0:
                        mixed_input.append(torch.cat((input_a[i][:length_a[i]], torch.tensor([102]).cuda(), input_b[i][:length_b[i]], torch.tensor([0]*(512-1-int(length_a[i])-int(length_b[i]))).cuda()), dim=0).unsqueeze(0))
                    else:
                        mixed_input.append(torch.cat((input_a[i][:length_a[i]], torch.tensor([102]), input_b[i][:length_b[i]], torch.tensor([0]*(512-1-int(length_a[i])-int(length_b[i])))), dim=0).unsqueeze(0))
                mixed_input = torch.cat(mixed_input, dim=0)
                logits = model(mixed_input, sent_size=512)

                #mixed_target = torch.clamp(target_a + target_b, max = 1)
                mixed = 0
                mixed_target = (target_a + target_b)/2
            else:
                mixed_input = input_a
                mixed_target = target_a
                logits = model(mixed_input, sent_size=max_seq_len)
                mixed = 1
        #使用SemiLoss计算损失
        if not train_aug:
          Lx, Lu, w, Lu2, w2 = criterion(logits[:batch_size], mixed_target[:batch_size], logits[-batch_size_2:],
                                       mixed_target[-batch_size_2:], logits[-batch_size_2:], epoch+batch_idx/args.val_iteration, mixed)
        else:
          Lx, Lu, w, Lu2, w2 = criterion(logits[:batch_size], mixed_target[:batch_size], logits[batch_size:-batch_size_2],
                                       mixed_target[batch_size:-batch_size_2], logits[-batch_size_2:], epoch+batch_idx/args.val_iteration, mixed)
        #如果使用mix
        if mix_ == 1:
            loss = Lx + w * Lu
        else:
            loss = Lx + w * Lu + w2 * Lu2
        if not train_aug:
          loss=Lx + w2 * Lu2 
        #max_grad_norm = 1.0
        # 梯度裁剪
        #torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        # scheduler.step()

        if batch_idx % 1000 == 0:
            print("epoch {}, step {}, loss {}, Lx {}, Lu {}, Lu2 {}".format(
                epoch, batch_idx, loss.item(), Lx.item(), Lu.item(), Lu2.item()))

In [ ]:
def initial(n_labels,checkpoint_name="/content/drive/My Drive/MixText/code/model/my_model.pt"):
  # 定义模型，设置优化器
  if n_gpu == 0:
        model = MixText(n_labels, args.mix_option, model=args.model)
  else:
        model = MixText(n_labels, args.mix_option, model=args.model).cuda()

  checkpoint=torch.load(checkpoint_name)

  model.load_state_dict(checkpoint)
  print("load model sucess!")
  model = nn.DataParallel(model)
  #优化器参数
  optimizer = AdamW(
        [
            {"params": model.module.bert.parameters(), "lr": args.lrmain},
            {"params": model.module.linear.parameters(), "lr": args.lrlast},
        ])
  #预热步数
  num_warmup_steps = math.floor(50)
  num_total_steps = args.val_iteration
  #是否动态更新学习率
  scheduler = None
  #WarmupConstantSchedule(optimizer, warmup_steps=num_warmup_steps)
  #训练损失函数，用在训练时
  train_criterion = SemiLoss()
  #交叉熵损失, 用在验证集和测试集
  criterion = nn.CrossEntropyLoss()
  return  model, optimizer,scheduler, train_criterion,criterion 

In [ ]:
train_prameters={
    "epochs":15,
    "batch_size":4,
    "batch_szie_u":12,
    "lrmax":0.00001,
    "lrlast":0.001,
    "gpu":1,
    "n_laneled":20,
    "un_labeled":5000,
    "val_iteration":100,
    "mix_option":True,
    "mix_method":0,
    "separate_mix":False,
    "co":False,
    "train_ug":False,
    "model":'bert-base-chinese',
    "data_path":"../data/NLP2",
    "mix_layers_set":[7,9,12],
    "alpha":0.4,
    "lambda_u":1,
    "T":0.5,
    "temp_change":1000000,
    "margin":0.7,
    "lambda_u_hinge":1
}
args=argment(train_prameters)

In [ ]:
##GPU相关设置, 设置为0表示不适用gpu
##os.environ['CUDA_VISIBLE_DEVICES'] = args.gpu
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
if n_gpu == 0:
    print("GPU 数量为0，使用CPU")
else:
    print("可以使用的GPU 数量: ", n_gpu)


best_acc = 0
total_steps = 0
flag = 0
print('是否要进行mix: ', args.mix_option)
print("Mix层集合: ", args.mix_layers_set)

可以使用的GPU 数量:  1
是否要进行mix:  True
Mix层集合:  [7, 9, 12]


In [ ]:
global best_acc

train_labeled_set, train_unlabeled_set, val_set, test_set, n_labels = get_data(
        "../data/NLP2",10, max_seq_len=256, model='bert-base-chinese', train_aug=False)
# 制作loader
labeled_trainloader = Data.DataLoader(
        dataset=train_labeled_set, batch_size=args.batch_size, shuffle=True)
unlabeled_trainloader = Data.DataLoader(
        dataset=train_unlabeled_set, batch_size=args.batch_size_u, shuffle=True)
val_loader = Data.DataLoader(
        dataset=val_set, batch_size=20, shuffle=False)
test_loader = Data.DataLoader(
        dataset=test_set, batch_size=20, shuffle=False)

100%|██████████| 109540/109540 [00:00<00:00, 429861.02B/s]


In [ ]:
from mixtext import *

In [ ]:
model, optimizer,scheduler, criterion=initial()
test_accs = []
#开始训练
for epoch in range(args.epochs):
        args.train_aug=False
        #调用train函数, 给定有标签数据，无标签数据，模型，优化器，损失函数，labels数量，是否数据增强
        train(labeled_trainloader, unlabeled_trainloader, model, optimizer,scheduler, train_criterion, epoch, n_labels, 256,args.train_aug)

        # scheduler.step()
        # _, train_acc = validate(labeled_trainloader, model,  criterion, epoch, mode='Train Stats')
        #print("epoch {}, train acc {}".format(epoch, train_acc))

        val_loss, val_acc = validate(val_loader, model, criterion, epoch, mode='Valid Stats')

        print("epoch {}, 验证集准确率 {}, 验证集损失 {}".format(epoch, val_acc, val_loss))

        print('Epoch: ', epoch)
        
        

print("完成训练")
print('最好的准去率')
print(best_acc)

print('测试准确率')
print(test_accs)


/content/drive/My Drive/MixText/code/pytransformers/optimization.py:166: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha) (Triggered internally at  /pytorch/torch/csrc/utils/python_arg_parser.cpp:882.)
  exp_avg.mul_(beta1).add_(1.0 - beta1, grad)


epoch 0, step 0, loss 0.1959916353225708, Lx 0.1959916353225708, Lu 0.08342268317937851, Lu2 0.13388267159461975
Sample some true labeles and predicted labels
tensor([3, 0, 1, 8, 3, 1, 4, 9, 3, 4, 0, 7, 6, 2, 0, 1, 0, 2, 4, 6],
       device='cuda:0')
tensor([3, 0, 2, 8, 3, 1, 4, 9, 3, 4, 0, 7, 6, 2, 0, 1, 0, 2, 4, 6],
       device='cuda:0')
epoch 0, 验证集准确率 0.8805748265609514, 验证集损失 0.4327510715682868
Epoch:  0
epoch 1, step 0, loss 1.4065368175506592, Lx 1.4024966955184937, Lu 0.05828262120485306, Lu2 0.060601331293582916
Sample some true labeles and predicted labels
tensor([6, 0, 1, 8, 3, 1, 4, 8, 3, 4, 0, 7, 6, 2, 0, 1, 0, 2, 4, 6],
       device='cuda:0')
tensor([3, 0, 2, 8, 3, 1, 4, 9, 3, 4, 0, 7, 6, 2, 0, 1, 0, 2, 4, 6],
       device='cuda:0')
epoch 1, 验证集准确率 0.8795837462834489, 验证集损失 0.4770932705746179
Epoch:  1
epoch 2, step 0, loss 0.5419385433197021, Lx 0.5403410792350769, Lu 0.08579348027706146, Lu2 0.011981050483882427
Sample some true labeles and predicted labels
tensor(

In [ ]:
torch.save(model.module.state_dict(), "/content/drive/My Drive/MixText/code/model/my_model3.pt")

NameError: ignored

In [ ]:
## make a predict 
model.eval()
outputs=[]
test_loader = Data.DataLoader(
        dataset=test_set, batch_size=30, shuffle=False)
for batch_idx, (inputs, length) in enumerate(test_loader):
            if n_gpu !=0:
                inputs = inputs.cuda()
            outputs.extend(model(inputs).detach().cpu().numpy())

In [ ]:
torch.max(torch.tensor(outputs), 1)

torch.return_types.max(values=tensor([5.0224, 6.1656, 5.1617,  ..., 6.2510, 5.2215, 5.1332]), indices=tensor([7, 9, 2,  ..., 9, 3, 7]))

In [ ]:
test_df=pd.read_csv("../data/NLP2/test_df.csv")

In [ ]:
train_df=pd.read_csv("../data/NLP2/train_valid.csv")

In [ ]:
_,test_df["num_label"]=torch.max(torch.tensor(outputs), 1)

In [ ]:
def name2label_conv(data):
    name2label={
        "财经":0,"房产":1,"家居":2,"教育":3,"科技":4,"时尚":5,"时政":6,"游戏":7,"娱乐":8,"体育":9
    }
    data["num_label"]=data["class_label"].apply(lambda x: name2label[x])
    return data

In [ ]:
def label2name_conv(data):
    label2name={0: '财经',
     1: '房产',
     2: '家居',
     3: '教育',
     4: '科技',
     5: '时尚',
     6: '时政',
     7: '游戏',
     8: '娱乐',
     9: '体育'}
    data["class_label"]=data["num_label"].apply(lambda x: label2name[x])
    return data

In [ ]:
def label2level_conv(data):
    level={"财经":"高风险",
           "房产":"中风险",
           "家居":"可公开",
           "教育":"低风险",
           "科技":"中风险",
           "时尚":"低风险",
           "时政":"高风险",
           "游戏":"低风险",
           "娱乐":"可公开",
           "体育":"可公开"}
    data["rank_label"]=data["class_label"].apply(lambda x: level[x])
    return data

In [ ]:
def submit(data,filename):
    data=label2name_conv(data)
    data=label2level_conv(data)
    data=data[["id","class_label","rank_label"]]
    data.to_csv(filename,index=False)

In [ ]:
train_df.iloc[1674]["content"]
#1674 ->游戏 可能部分添加错误的标签造成了当前的错误分类

In [ ]:
submit(test_df,"mixText.csv")

In [ ]:
## 自监督训练 


In [ ]:
model, optimizer,scheduler, train_criterion,criterion=initial( 10,"/content/drive/My Drive/MixText/code/model/self_train_model9.pt")

100%|██████████| 411577189/411577189 [00:12<00:00, 31938314.40B/s]


load model sucess!


In [ ]:
## Make predict
def text_filter(sentence: str) -> str:
        """
        过滤掉非汉字和标点符号和非数字
        :param sentence:
        :return:
        """
        line = sentence.replace('\n', '。')
        # 过滤掉非汉字和标点符号和非数字
        linelist = [word for word in line if
                    word >= u'\u4e00' and word <= u'\u9fa5' or word in ['，', '。', '？', '！',
                                                                        '：'] or word.isdigit()]
        return ''.join(linelist)
def fliter_data(data):
      data['content']=data['content'].astype(str)
      data["text"]=data["content"].apply(lambda x:text_filter(x))
      return data 

test_df=pd.read_csv("../data/NLP2/0test_df.csv")
test_df=fliter_data(test_df)
tokenizer = BertTokenizer.from_pretrained('bert-base-chinese')
test_set = loader_unlabeled(list(test_df["text"]),tokenizer, max_seq_len=256, train_aug=False)
test_loader = Data.DataLoader(
        dataset=test_set, batch_size=30, shuffle=False)
model.eval()
outputs=[]
with torch.no_grad():
  for batch_idx, (inputs, length) in enumerate(test_loader):
            if n_gpu !=0:
                inputs = inputs.cuda()
            outputs.extend(model(inputs).detach().cpu().numpy())
_,test_df["num_label"]=torch.max(torch.tensor(outputs), 1)

submit(test_df,"mixText_self_train9.csv")

100%|██████████| 109540/109540 [00:00<00:00, 831953.93B/s]


In [ ]:
def self_train( model,optimizer,scheduler,train_criterion,criterion,data_path,step,nlabels,
               checkpoint_name=None,max_seq_len=256, model_type='bert-base-chinese', train_aug=False):
  
  tokenizer = BertTokenizer.from_pretrained(model_type)

  def text_filter(sentence: str) -> str:
        """
        过滤掉非汉字和标点符号和非数字
        :param sentence:
        :return:
        """
        line = sentence.replace('\n', '。')
        # 过滤掉非汉字和标点符号和非数字
        linelist = [word for word in line if
                    word >= u'\u4e00' and word <= u'\u9fa5' or word in ['，', '。', '？', '！',
                                                                        '：'] or word.isdigit()]
        return ''.join(linelist)
  def fliter_data(data):
      data['content']=data['content'].astype(str)
      data["text"]=data["content"].apply(lambda x:text_filter(x))
      return data 


  train_labeled=pd.read_csv(data_path+"/"+str(step)+"train.csv")
  valid_labeled=pd.read_csv(data_path+"/valid.csv")
  train_ul=pd.read_csv(data_path+"/"+str(step)+"train_ul.csv")
  test_df=pd.read_csv(data_path+"/"+str(step)+"test_df.csv")



  train_labeled=fliter_data(train_labeled)
  valid_labeled=fliter_data(valid_labeled)
  train_ul=fliter_data(train_ul)
  test_df=fliter_data(test_df)

  total_unlabeled=pd.concat([train_ul,test_df])

  train_labeled_set = loader_labeled(list(train_labeled["text"]),list(train_labeled["num_label"]), tokenizer, max_seq_len, train_aug)
  train_unlabeled_set = loader_unlabeled(list(total_unlabeled["text"]),tokenizer, max_seq_len, train_aug)
  train_ul_set= loader_unlabeled(list(train_ul["text"]),tokenizer, max_seq_len, train_aug)
  val_set = loader_labeled(list(valid_labeled["text"]),list(valid_labeled["num_label"]), tokenizer, max_seq_len, train_aug)
  test_set = loader_unlabeled(list(test_df["text"]),tokenizer, max_seq_len, train_aug)

  # 制作loader
  labeled_trainloader = Data.DataLoader(
        dataset=train_labeled_set, batch_size=args.batch_size, shuffle=True)
  unlabeled_trainloader = Data.DataLoader(
        dataset=train_unlabeled_set, batch_size=args.batch_size_u, shuffle=True)
  ## use for self_training 
  unlabeled_trainloader2 = Data.DataLoader(
        dataset=train_ul_set, batch_size=args.batch_size_u, shuffle=False)
  val_loader = Data.DataLoader(
        dataset=val_set, batch_size=20, shuffle=False)
  test_loader = Data.DataLoader(
        dataset=test_set, batch_size=20, shuffle=False)

  #read model from prev train 
  #checkpoint=torch.load(checkpoint_name)
  #model.load_state_dict(checkpoint)

  #再次训练
  for epoch in range(args.epochs):
        args.train_aug=False
        #调用train函数, 给定有标签数据，无标签数据，模型，优化器，损失函数，labels数量，是否数据增强
        train(labeled_trainloader, unlabeled_trainloader, model, optimizer,scheduler, train_criterion, epoch, nlabels, 256,args.train_aug)

        # scheduler.step()
        # _, train_acc = validate(labeled_trainloader, model,  criterion, epoch, mode='Train Stats')
        #print("epoch {}, train acc {}".format(epoch, train_acc))

        val_loss, val_acc = validate(val_loader, model, criterion, epoch, mode='Valid Stats')

        print("epoch {}, 验证集准确率 {}, 验证集损失 {}".format(epoch, val_acc, val_loss))

        print('Epoch: ', epoch)
        
  ## make predict 
  append_df=pd.DataFrame([])
  ## make a predict 
  outputs=[]
  for batch_idx, (inputs, length) in enumerate(test_loader):
            if n_gpu !=0:
                inputs = inputs.cuda()
            outputs.extend(model(inputs).detach().cpu().numpy())
  ## add the high probility sample to train_data

  ## 选择低熵样本
  model.eval()
  with torch.no_grad():
   outputs_u=torch.tensor(outputs)
   test_df["entropy"]=torch.sum(-F.softmax(outputs_u, dim=1) * F.log_softmax(outputs_u, dim=1), dim=1).cpu().numpy()
   _,test_df["num_label"]=torch.max(outputs_u, 1)
   test_df=test_df.sort_values("entropy").reset_index(drop=True)
   append_df1=test_df.iloc[:int(0.1 * len(test_df)),]
   test_df=test_df.iloc[int(0.1 * len(test_df)):,]
   append_df=pd.concat([append_df,append_df1])

  outputs=[]
  for batch_idx, (inputs, length) in enumerate(unlabeled_trainloader2):
            if n_gpu !=0:
                inputs = inputs.cuda()
            outputs.extend(model(inputs).detach().cpu().numpy())
  ## add the high probility sample to train_data

  ## 选择低熵样本
  model.eval()
  with torch.no_grad():
    outputs_u=torch.tensor(outputs)
    train_ul["entropy"]=torch.sum(-F.softmax(outputs_u, dim=1) * F.log_softmax(outputs_u, dim=1), dim=1).cpu().numpy()
    _,train_ul["num_label"]=torch.max(outputs_u, 1)

    train_ul=train_ul.sort_values("entropy").reset_index(drop=True)
    append_df2=train_ul.iloc[:int(0.1 * len(train_ul)),]
    train_ul=train_ul.iloc[int(0.1 * len(train_ul)):,]
    append_df=pd.concat([append_df,append_df2])

  ## get new train_labeled,train_ul,test_df 
  train_labeled=pd.concat([train_labeled,append_df])
  train_ul=train_ul
  test_df=test_df 

  train_labeled.to_csv(data_path+"/"+str(step+1)+"train.csv",index=False)
  train_ul.to_csv(data_path+"/"+str(step+1)+"train_ul.csv",index=False)
  test_df.to_csv(data_path+"/"+str(step+1)+"test_df.csv",index=False)

  #save model 
  torch.save(model.module.state_dict(), "/content/drive/My Drive/MixText/code/model/self_train_model"+str(step)+".pt")
  

In [ ]:
self_train( model, optimizer,scheduler, train_criterion,criterion,"../data/NLP2",0,10,
               checkpoint_name=None,max_seq_len=256, model_type='bert-base-chinese', train_aug=False)

100%|██████████| 109540/109540 [00:00<00:00, 2748002.66B/s]
/content/drive/My Drive/MixText/code/pytransformers/optimization.py:166: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha) (Triggered internally at  /pytorch/torch/csrc/utils/python_arg_parser.cpp:882.)
  exp_avg.mul_(beta1).add_(1.0 - beta1, grad)


epoch 0, step 0, loss 0.06372619420289993, Lx 0.06372619420289993, Lu 0.04901032894849777, Lu2 0.0
Sample some true labeles and predicted labels
tensor([0, 3, 7, 1, 0, 0, 5, 5, 0, 3, 2, 1, 2, 6, 1, 4, 1, 0, 3, 3],
       device='cuda:0')
tensor([0, 3, 7, 1, 0, 0, 5, 5, 4, 3, 2, 1, 2, 6, 1, 4, 1, 0, 3, 3],
       device='cuda:0')
epoch 0, 验证集准确率 0.8998512642538423, 验证集损失 0.41528084906289997
Epoch:  0
epoch 1, step 0, loss 0.16093216836452484, Lx 0.15840516984462738, Lu 0.0442013144493103, Lu2 0.03790486603975296
Sample some true labeles and predicted labels
tensor([0, 3, 7, 1, 2, 0, 5, 5, 4, 3, 2, 1, 2, 6, 1, 4, 1, 0, 3, 3],
       device='cuda:0')
tensor([0, 3, 7, 1, 0, 0, 5, 5, 4, 3, 2, 1, 2, 6, 1, 4, 1, 0, 3, 3],
       device='cuda:0')
epoch 1, 验证集准确率 0.8760535448686168, 验证集损失 0.4716260166793515
Epoch:  1
epoch 2, step 0, loss 0.0760982558131218, Lx 0.0760982558131218, Lu 0.039917901158332825, Lu2 0.0
Sample some true labeles and predicted labels
tensor([0, 3, 7, 2, 0, 0, 5, 5, 4, 3

In [ ]:
self_train( model, optimizer,scheduler, train_criterion,criterion,"../data/NLP2",1,10,
               checkpoint_name=None,max_seq_len=256, model_type='bert-base-chinese', train_aug=False)

epoch 0, step 0, loss 0.5419917106628418, Lx 0.5419917106628418, Lu 0.04861962050199509, Lu2 0.008538400754332542
Sample some true labeles and predicted labels
tensor([0, 1, 7, 1, 0, 0, 5, 5, 4, 3, 2, 1, 2, 6, 1, 4, 1, 0, 3, 3],
       device='cuda:0')
tensor([0, 3, 7, 1, 0, 0, 5, 5, 4, 3, 2, 1, 2, 6, 1, 4, 1, 0, 3, 3],
       device='cuda:0')
epoch 0, 验证集准确率 0.8839861179970253, 验证集损失 0.45276778645307403
Epoch:  0
epoch 1, step 0, loss 0.07909555733203888, Lx 0.07672975212335587, Lu 0.07046167552471161, Lu2 0.035487063229084015
Sample some true labeles and predicted labels
tensor([0, 3, 7, 1, 0, 0, 9, 5, 4, 3, 2, 1, 2, 6, 1, 4, 1, 0, 3, 3],
       device='cuda:0')
tensor([0, 3, 7, 1, 0, 0, 5, 5, 4, 3, 2, 1, 2, 6, 1, 4, 1, 0, 3, 3],
       device='cuda:0')
epoch 1, 验证集准确率 0.8750619732275657, 验证集损失 0.47981369800507345
Epoch:  1
epoch 2, step 0, loss 0.5256173014640808, Lx 0.5256173014640808, Lu 0.03386594355106354, Lu2 0.0
Sample some true labeles and predicted labels
tensor([0, 3, 7, 6,

In [ ]:
self_train( model, optimizer,scheduler, train_criterion,criterion,"../data/NLP2",2,10,
               checkpoint_name=None,max_seq_len=256, model_type='bert-base-chinese', train_aug=False)

epoch 0, step 0, loss 0.21014457941055298, Lx 0.21014457941055298, Lu 0.031090810894966125, Lu2 0.0
Sample some true labeles and predicted labels
tensor([0, 3, 7, 1, 0, 0, 5, 5, 0, 3, 2, 1, 2, 6, 1, 4, 1, 0, 3, 3],
       device='cuda:0')
tensor([0, 3, 7, 1, 0, 0, 5, 5, 4, 3, 2, 1, 2, 6, 1, 4, 1, 0, 3, 3],
       device='cuda:0')
epoch 0, 验证集准确率 0.8834903321764997, 验证集损失 0.4970236609389301
Epoch:  0
epoch 1, step 0, loss 0.05465487390756607, Lx 0.05465487390756607, Lu 0.043505966663360596, Lu2 0.0
Sample some true labeles and predicted labels
tensor([0, 3, 7, 1, 0, 0, 5, 5, 4, 3, 2, 1, 2, 6, 1, 4, 1, 0, 3, 3],
       device='cuda:0')
tensor([0, 3, 7, 1, 0, 0, 5, 5, 4, 3, 2, 1, 2, 6, 1, 4, 1, 0, 3, 3],
       device='cuda:0')
epoch 1, 验证集准确率 0.887456618740704, 验证集损失 0.4873255893004431
Epoch:  1
epoch 2, step 0, loss 0.2523789405822754, Lx 0.2510702610015869, Lu 0.11290355771780014, Lu2 0.009814992547035217
Sample some true labeles and predicted labels
tensor([0, 3, 7, 1, 0, 0, 5, 5, 0, 

In [ ]:
self_train( model, optimizer,scheduler, train_criterion,criterion,"../data/NLP2",3,10,
               checkpoint_name=None,max_seq_len=256, model_type='bert-base-chinese', train_aug=False)

/content/drive/My Drive/MixText/code/pytransformers/optimization.py:166: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha) (Triggered internally at  /pytorch/torch/csrc/utils/python_arg_parser.cpp:882.)
  exp_avg.mul_(beta1).add_(1.0 - beta1, grad)


epoch 0, step 0, loss 0.5243107080459595, Lx 0.5243107080459595, Lu 0.17948952317237854, Lu2 0.0
Sample some true labeles and predicted labels
tensor([0, 3, 7, 1, 0, 0, 5, 5, 0, 3, 2, 1, 2, 6, 1, 4, 1, 0, 3, 3],
       device='cuda:0')
tensor([0, 3, 7, 1, 0, 0, 5, 5, 4, 3, 2, 1, 2, 6, 1, 4, 1, 0, 3, 3],
       device='cuda:0')
epoch 0, 验证集准确率 0.8790282597917699, 验证集损失 0.5058592915069543
Epoch:  0
epoch 1, step 0, loss 0.4281255006790161, Lx 0.4281255006790161, Lu 0.04600412771105766, Lu2 0.0
Sample some true labeles and predicted labels
tensor([0, 3, 7, 1, 0, 0, 5, 5, 4, 3, 2, 1, 2, 6, 1, 4, 1, 0, 3, 3],
       device='cuda:0')
tensor([0, 3, 7, 1, 0, 0, 5, 5, 4, 3, 2, 1, 2, 6, 1, 4, 1, 0, 3, 3],
       device='cuda:0')
epoch 1, 验证集准确率 0.8904313336638572, 验证集损失 0.4976002755706477
Epoch:  1
epoch 2, step 0, loss 0.6373498439788818, Lx 0.6373498439788818, Lu 0.030975280329585075, Lu2 0.0
Sample some true labeles and predicted labels
tensor([0, 3, 7, 1, 0, 0, 5, 5, 4, 3, 2, 1, 2, 6, 1, 4, 

In [ ]:
self_train( model, optimizer,scheduler, train_criterion,criterion,"../data/NLP2",4,10,
               checkpoint_name=None,max_seq_len=256, model_type='bert-base-chinese', train_aug=False)

epoch 0, step 0, loss 0.33009451627731323, Lx 0.33009451627731323, Lu 0.044966455549001694, Lu2 0.015052437782287598
Sample some true labeles and predicted labels
tensor([0, 3, 7, 1, 0, 0, 9, 5, 4, 3, 2, 1, 2, 6, 1, 4, 1, 0, 3, 3],
       device='cuda:0')
tensor([0, 3, 7, 1, 0, 0, 5, 5, 4, 3, 2, 1, 2, 6, 1, 4, 1, 0, 3, 3],
       device='cuda:0')
epoch 0, 验证集准确率 0.8790282597917699, 验证集损失 0.5588520097411914
Epoch:  0
epoch 1, step 0, loss 0.8526284694671631, Lx 0.8526284694671631, Lu 0.031071756035089493, Lu2 0.0
Sample some true labeles and predicted labels
tensor([0, 3, 7, 1, 0, 0, 9, 5, 4, 3, 2, 1, 2, 6, 1, 4, 1, 0, 3, 3],
       device='cuda:0')
tensor([0, 3, 7, 1, 0, 0, 5, 5, 4, 3, 2, 1, 2, 6, 1, 4, 1, 0, 3, 3],
       device='cuda:0')
epoch 1, 验证集准确率 0.8815071888943976, 验证集损失 0.5478341435338521
Epoch:  1
epoch 2, step 0, loss 0.5372110605239868, Lx 0.5372110605239868, Lu 0.01795952580869198, Lu2 0.0
Sample some true labeles and predicted labels
tensor([0, 3, 7, 6, 0, 0, 9, 5, 4, 3

In [ ]:
self_train( model, optimizer,scheduler, train_criterion,criterion,"../data/NLP2",5,10,
               checkpoint_name=None,max_seq_len=256, model_type='bert-base-chinese', train_aug=False)

epoch 0, step 0, loss 0.22489199042320251, Lx 0.22489199042320251, Lu 0.02667880430817604, Lu2 0.0
Sample some true labeles and predicted labels
tensor([0, 3, 7, 1, 0, 0, 9, 5, 4, 3, 2, 1, 2, 2, 1, 4, 1, 0, 3, 3],
       device='cuda:0')
tensor([0, 3, 7, 1, 0, 0, 5, 5, 4, 3, 2, 1, 2, 6, 1, 4, 1, 0, 3, 3],
       device='cuda:0')
epoch 0, 验证集准确率 0.8750619732275657, 验证集损失 0.5648630433667398
Epoch:  0
epoch 1, step 0, loss 0.1725761890411377, Lx 0.1725761890411377, Lu 0.03718619793653488, Lu2 0.0
Sample some true labeles and predicted labels
tensor([0, 3, 7, 1, 0, 0, 9, 5, 4, 3, 2, 1, 2, 2, 1, 4, 1, 0, 3, 3],
       device='cuda:0')
tensor([0, 3, 7, 1, 0, 0, 5, 5, 4, 3, 2, 1, 2, 6, 1, 4, 1, 0, 3, 3],
       device='cuda:0')
epoch 1, 验证集准确率 0.880019831432821, 验证集损失 0.5291146788456148
Epoch:  1
epoch 2, step 0, loss 0.5069714784622192, Lx 0.5054399967193604, Lu 0.036388788372278214, Lu2 0.011486157774925232
Sample some true labeles and predicted labels
tensor([0, 3, 7, 1, 0, 0, 9, 5, 4, 3, 

In [ ]:
self_train( model, optimizer,scheduler, train_criterion,criterion,"../data/NLP2",6,10,
               checkpoint_name=None,max_seq_len=256, model_type='bert-base-chinese', train_aug=False)

epoch 0, step 0, loss 0.19834604859352112, Lx 0.19834604859352112, Lu 0.031504444777965546, Lu2 0.011127566918730736
Sample some true labeles and predicted labels
tensor([0, 3, 7, 4, 0, 0, 5, 5, 4, 3, 2, 1, 2, 6, 1, 4, 1, 0, 3, 3],
       device='cuda:0')
tensor([0, 3, 7, 1, 0, 0, 5, 5, 4, 3, 2, 1, 2, 6, 1, 4, 1, 0, 3, 3],
       device='cuda:0')
epoch 0, 验证集准确率 0.8790282597917699, 验证集损失 0.5792128899507317
Epoch:  0
epoch 1, step 0, loss 0.013078122399747372, Lx 0.013078122399747372, Lu 0.18781745433807373, Lu2 0.0
Sample some true labeles and predicted labels
tensor([0, 3, 7, 4, 0, 0, 5, 5, 4, 3, 2, 1, 2, 6, 1, 4, 1, 0, 3, 3],
       device='cuda:0')
tensor([0, 3, 7, 1, 0, 0, 5, 5, 4, 3, 2, 1, 2, 6, 1, 4, 1, 0, 3, 3],
       device='cuda:0')
epoch 1, 验证集准确率 0.8696083292017849, 验证集损失 0.6456562662197034
Epoch:  1
epoch 2, step 0, loss 0.012936396524310112, Lx 0.011392618529498577, Lu 0.12902066111564636, Lu2 0.011578336358070374
Sample some true labeles and predicted labels
tensor([0, 3

In [ ]:
self_train( model, optimizer,scheduler, train_criterion,criterion,"../data/NLP2",7,10,
               checkpoint_name=None,max_seq_len=256, model_type='bert-base-chinese', train_aug=False)

epoch 0, step 0, loss 0.36285215616226196, Lx 0.36285215616226196, Lu 0.020751729607582092, Lu2 0.003670449135825038
Sample some true labeles and predicted labels
tensor([0, 3, 7, 1, 1, 0, 5, 5, 4, 3, 2, 1, 2, 3, 1, 4, 1, 0, 3, 3],
       device='cuda:0')
tensor([0, 3, 7, 1, 0, 0, 5, 5, 4, 3, 2, 1, 2, 6, 1, 4, 1, 0, 3, 3],
       device='cuda:0')
epoch 0, 验证集准确率 0.8309370352007932, 验证集损失 0.7790100058752177
Epoch:  0
epoch 1, step 0, loss 0.049982283264398575, Lx 0.049982283264398575, Lu 0.046333372592926025, Lu2 0.0
Sample some true labeles and predicted labels
tensor([0, 3, 7, 1, 0, 0, 5, 5, 4, 3, 2, 1, 2, 3, 1, 4, 1, 0, 3, 3],
       device='cuda:0')
tensor([0, 3, 7, 1, 0, 0, 5, 5, 4, 3, 2, 1, 2, 6, 1, 4, 1, 0, 3, 3],
       device='cuda:0')
epoch 1, 验证集准确率 0.8542389687654933, 验证集损失 0.7468219656528639
Epoch:  1
epoch 2, step 0, loss 0.740126371383667, Lx 0.740126371383667, Lu 0.021113257855176926, Lu2 0.0
Sample some true labeles and predicted labels
tensor([0, 3, 7, 1, 0, 0, 5, 5, 4

In [ ]:
self_train( model, optimizer,scheduler, train_criterion,criterion,"../data/NLP2",8,10,
               checkpoint_name=None,max_seq_len=256, model_type='bert-base-chinese', train_aug=False)

epoch 0, step 0, loss 0.5420262813568115, Lx 0.5420262813568115, Lu 0.01122494786977768, Lu2 0.0
Sample some true labeles and predicted labels
tensor([0, 3, 7, 4, 0, 0, 5, 5, 4, 3, 2, 1, 2, 6, 1, 4, 1, 0, 3, 3],
       device='cuda:0')
tensor([0, 3, 7, 1, 0, 0, 5, 5, 4, 3, 2, 1, 2, 6, 1, 4, 1, 0, 3, 3],
       device='cuda:0')
epoch 0, 验证集准确率 0.865146256817055, 验证集损失 0.6948953244353085
Epoch:  0
epoch 1, step 0, loss 0.35414549708366394, Lx 0.35414549708366394, Lu 0.03165385499596596, Lu2 0.0
Sample some true labeles and predicted labels
tensor([0, 3, 7, 4, 0, 0, 5, 5, 4, 3, 2, 1, 2, 3, 1, 4, 1, 0, 3, 3],
       device='cuda:0')
tensor([0, 3, 7, 1, 0, 0, 5, 5, 4, 3, 2, 1, 2, 6, 1, 4, 1, 0, 3, 3],
       device='cuda:0')
epoch 1, 验证集准确率 0.866137828458106, 验证集损失 0.6569095668879309
Epoch:  1
epoch 2, step 0, loss 0.01323101669549942, Lx 0.011589540168642998, Lu 0.06636351346969604, Lu2 0.012311076745390892
Sample some true labeles and predicted labels
tensor([0, 3, 7, 1, 0, 0, 5, 5, 4, 3,

In [ ]:
self_train( model, optimizer,scheduler, train_criterion,criterion,"../data/NLP2",9,10,
               checkpoint_name=None,max_seq_len=256, model_type='bert-base-chinese', train_aug=False)